## The Clark-Wright Savings Algorithm

The savings algorithm aims to find a solution to the Vechile Routing Problem (VRP). It starts off with a trivial solution where there are $n$ vehicles which deliver to $n$ nodes. It then merges routes based on the highest possible savings per merge. The idea is to be able to start with the image on the left and move to the image on the right. 

<img src=img/clark.png width="300" height="200" / >


Basically the algorithm can be understood as follows: 
<ol>
  <li>Start off with a trivial solution to the problem. That is, have one vechicle travel from the depot to each node.</li>
  <li>Create a savings matrix such that s(i, j) = distance(i, depot) + distance(depot, j) - distance(i, j)</li>
  <li>Sort your savings matrix so you consider the pairs with the highest savings first</li>
  <li>Merge those nodes if they meet all the feasibility constraints</li>
</ol>         


TODO

4. For savings list, use values that would occur if you merged closest carryin/prep 

# Reading in Data

In [58]:
import random
from typing import Callable, List
import pandas as pd
import numpy  as np
from graph import Graph
from segment import Segment

In [59]:
# Reading Data
prep_carry_matches = pd.read_csv('data/prep-carry-matches2.csv')
delivery_times     = pd.read_csv('data/delivery-times2.csv')
timematrix         = pd.read_csv('data/time_matrix.csv')
prep_carry_matches.replace({np.nan: None}, inplace=True)

# Getting Ready for Savings Algorithm
We need to create some functions and prepare the data before we can get started.

## Creating Segments
Here, we're building the segments based on the given matching guidelines. That is, if some prep site delivers to some carry in sites, that's one segment.

In [64]:
# Defining all the segments. Each segment has one prep site, one/two carry-in sites, and one timewindow
#
# Ex: Segment 0 -> Prep Site = 450 | Carry-Ins = [604,] | Time-Window = (6:15, 7:15)
#     Segment 1 -> Prep Site = 450 | Carry-Ins = [604,] | Time-Windwo = (9:30, 10:30)
segments = []
for i in prep_carry_matches.values.tolist():
    obj = Segment(int(i[0]), int(i[1]), int(i[2]) if i[2] else None)
    segments.append(obj)

## Creating Distance Function
This is to help us later. We have a csv file with the times from each school to another. So, to simplify accesing these times, we're creating a function, `distance(src, dst)` which takes in two school's three digit codes and returns the time it takes to go from one to the next.

In [61]:
# Creating a distance function from the distance matrix
# Given two school's 3 digit codes, it returns the distance in time between them
# Ex: distance(981, 604) = 35.736
origin      = list(timematrix['Origin_School'])
destination = list(timematrix['Dest_School'])
times       = list(timematrix['Total_TravelTime'])

pairs = zip(origin, destination, times)
distance_dictionary = {}
for src, dst, time in pairs:
    distance_dictionary[(src, dst)] = time

def distance(src_, dst_):
    return distance_dictionary[(src_, dst_)]

## Creating Time Windows


This part is a bit trickier. We need to create time windows for each of the segments. These are going to be used when we execute the savings algorithm to make sure we compute feasible routes. The time window for each segment is defined to be 

~~~
(start = earliest time any carryin is visted, end = earliest time any carryin is left)
~~~

In [62]:
# Given a carryin site, it returns the scheulde for it's breakfast and lunch
def time_lookup(carryin_: int):
    for i in delivery_times.values:
        if i[0] == carryin_:
            return list(i)
    raise Exception(f'carryin site {carryin_} not found')

In [63]:
# Given two start and end times as strings, it converts the earliest of each to datetime format and returns it.
def strict_window(start: str, end:str, start2 = '', end2 = ''):
    if start == 'DIA':
        start = '6:00 AM'
    if start2 == 'DIA':
        start2 = '6:00 AM'

    if start2 == '' and end2 == '':
        return (pd.to_datetime(start), pd.to_datetime(end))
    start  = pd.to_datetime(start)
    start2 = pd.to_datetime(start2)
    end    = pd.to_datetime(end)
    end2   = pd.to_datetime(end2)
    if start2 < start:
        start = pd.to_datetime(start2)

    if end2 < end2:
        end = pd.to_datetime(end2)

    return(start, end)

In [65]:
# Given all segments (assuming they have one or two carry-in sites), it builds time windows for those segments 
# It returns a dictionary which maps segments to time windows
def window_builder(segments_: List[Segment]):
    seen = {}
    windows = {}
    visited = {}
    for seg in segments_:
        for carry in [seg.carry1, seg.carry2]:
            if carry not in visited:
                visited[carry] = 1
            else:
                visited[carry] += 1

    # For every segment, find the time constraints for the first carryin according to delivery_times.csv
    for indx,seg in enumerate(segments_):
        i = time_lookup(seg.carry1)
        if visited[i[0]] == 2:
            start = i[1]
            end   = i[2]
            visited[i[0]] -= 1
        else:
            start = i[3]
            end   = i[4] 
        # If the next carry in is 0 (meaning there is only one carry in) then just add the window and move on
        if seg.carry2 is None: 
            windows[indx] = strict_window(start, end)
            continue

        # If there is a second carry-in site, explore that one as well
        i = time_lookup(seg.carry2)
        if visited[i[0]] == 2:
            start2 = i[1]
            end2   = i[2]
            visited[i[0]] -= 1
        else:
            start2 = i[3]
            end2   = i[4] 
        # For segments which have two carry ins - find the approriate start and end time of the window
        windows[indx] = strict_window(start, end, start2, end2)
    return windows

## Creating Service Times
Here, we need to determine how long it takes to service every segment. 

- If we're visiting just one carryin site it's going to be 

        distance(prep, carry) + load + unload

 - If we're visiting two carryin sites, then it'll be 

        min(distance(prep,carry1), distance(prep,carry2)) + distance(carry1, carry2) + load + 2*unload

What does that mean? It's saying the service time is the time it takes to go from the prep site to the closest carryin site and then going to the next carry in. We also have to account for the time it takes to load and unload food. We have to load once for every segment and unload 

In [67]:
# Calculates the service time for a segment given loading and unloading times
def service_time_calc(segment_: Segment, load_: int, unload_: int):
    to_car_1 = distance(segment_.prep, segment_.carry1)
    if segment_.carry2 is None:
        return to_car_1 + load_ + unload_
    to_car_2 = distance(segment_.prep, segment_.carry2)
    return min(to_car_1, to_car_2) + load_ + 2*unload_

## Updating Segments 
Up until now, we did a lot of prep work to make functions that will fill in the fields for our segments. Here, we're using those functions so each `Segment()` object has its corresponding time window and service times. 

In [71]:
# Adding time_window and service_time fields to every segment
def seg_builder(segments_: List[Segment], load_:int, unload_:int):
    seg_windows = window_builder(segments_)
    for indx,seg in enumerate(segments_):
        seg.time_window = seg_windows[indx]
        seg.service_time = service_time_calc(seg, load_, unload_)

# Modified Savings Algorithm 
All the prep work is in place. We're ready to code the modified savings algorithm.

## Savings List
First up, we need to create a list of savings. That is, we need to figure out how much time we can save by merging two segments and putting them on the same route

In [69]:
# Creates a savings list for all segment pairs 
def savings_generator(segments_: List[Segment], depot_: int):
    savings = []
    for indx1, seg1 in enumerate(segments_):
        for indx2,seg2 in enumerate(segments_):
            if seg1 is seg2:
                continue
            save = distance(seg1.prep, depot_) + distance(depot_, seg2.prep) - distance(seg1.prep, seg2.prep)
            savings.append((indx1, indx2, save))
    savings.sort(key=lambda tup: tup[2])
    return savings

## Savings algorithm
Here, we'll actually run the algorithm. The code is super short, but there's a lot going on under the hood within the `Graph()` class. You can see how the class is implemented in the `graph.py` file.

In [70]:
def savings_algorithm(segments_: List[Segment], distance_: Callable[[int, int], float]):
    routes = Graph(segments_, distance_)
    savings = savings_generator(segments, 0)
    while savings:
        topop = random.randint(len(savings)-5, len(savings)-1)
        topop = topop if topop >=0 else 0
        highest_savings = savings.pop()
        routes.merge(highest_savings[0], highest_savings[1])
    return routes

In [72]:
def main():
    seg_builder(segments, 10, 10)
    routes = []
    y = savings_algorithm(segments, distance)
    for i in y.routes:
        if y.routes[i] not in routes:
            routes.append(y.routes[i])
    toprint = ""
    start   = 1
    for leg in routes:
        for indx in leg:
            seg = segments[indx]
            toprint += f"{seg.prep} -> {seg.carry1} -> {seg.carry2 if seg.carry2 else ''} -> "
        print(start, toprint)
        print("\n")
        start += 1
        toprint = ""
main()

NameError: name 'load' is not defined